# <font color='Indigo'>Gravitational Wave Generation Array</font>

A Phase Array of dumbells can make a detectable signal...

#### To do:
1. Calculate the dumbell parameters for given mass and frequency
1. How many dumbells?
1. Far-field radiation pattern from many radiators.
1. Beamed GW won't be a plane wave. So what?
1. How much energy is lost to keep it spinning?
1. How do we levitate while spinning?

##### Python Style Guide (don't write crappy obfuscated code):
- http://docs.python-guide.org/en/latest/writing/style/
- http://python.net/~goodger/projects/pycon/2007/idiomatic/handout.html

##### Run ipython notebook with a remote kernel
- https://github.com/gammapy/gammapy/wiki/IPython-Notebook-with-Remote-Kernel

## <font color='Navy'>Imports, settings, and constants</font>

In [1]:
%matplotlib inline
#from __future__ import division
#from __future__ import print_function
import numpy as np
#import matplotlib as mpl
import matplotlib.pyplot as plt
#import multiprocessing as mproc
#import scipy.signal as sig
import scipy.constants as scc
#import scipy.special as scsp
#import sys, time
from scipy.io import loadmat

# http://www.astropy.org/astropy-tutorials/Quantities.html
# http://docs.astropy.org/en/stable/constants/index.html
from astropy import constants as ascon

# Update the matplotlib configuration parameters:
plt.rcParams['figure.figsize'] = [10,8]
plt.rcParams.update({'lines.linewidth': 2,
                     'text.usetex': False,
                     'font.size': 18,
                     'font.family': 'serif'})

def setGrid(ax):
    ax.grid(which='major', alpha=0.6)
    ax.grid(which='major', linestyle='solid', alpha=0.6)
    
cList = [(0, 0.1, 0.9),
         (0.9, 0, 0),
         (0, 0.7, 0),
         (0, 0.8, 0.8),
         (1.0, 0, 0.9),
         (0.8, 0.8, 0),
         (1, 0.5, 0),
         (0.5, 0.5, 0.5),
         (0.4, 0, 0.5),
         (0, 0, 0),
         (0.3, 0, 0),
         (0, 0.3, 0)]

G      = scc.G              # N * m**2 / kg**2; gravitational constant
c      = scc.c

## Terrestrial Dumbell (Current Tech)

In [55]:
sigma_yield = 4700e6       # ultimate tensile strength of S-glass bundle [Pa]
m_dumb = 100               # mass of the dumbell end [kg]
L_dumb = 3                 # Length of the dumbell [m]
r_dumb = 40e-2             # radius of the dumbell rod [m]

rho_pb = 11.34e3        # density of lead [kg/m^3]
r_ball = ((m_dumb / rho_pb)/(4/3 * np.pi))**(1/3)


f_rot = 1e3 / 2
lamduh = c / f_rot

v_dumb = 2*np.pi*(L_dumb/2) * f_rot
a_dumb = v_dumb**2 / (L_dumb / 2)
F = a_dumb * m_dumb

stress = F / (np.pi * r_dumb**2)

print('Ball radius is ' + '{:0.2f}'.format(r_ball) + ' m')
print('Acceleration of ball = ' + '{:0.2g}'.format(a_dumb) + ' m/s**2')
print('Stress = ' + '{:0.2f}'.format(stress/sigma_yield) + 'x Yield Stress')

Ball radius is 0.13 m
Acceleration of ball = 1.5e+07 m/s**2
Stress = 0.63x Yield Stress


#### Futuristic Dumbell

In [64]:
sigma_yield = 5000e9    # ultimate tensile strength of ??? [Pa]
m_f = 1000               # mass of the dumbell end [kg]
L_f = 3000               # Length of the dumbell [m]
r_f = 40            # radius of the dumbell rod [m]

rho_pb = 11.34e3        # density of lead [kg/m^3]
r_b = ((m_dumb / rho_pb)/(4/3 * np.pi))**(1/3)


f_f = 37e3 / 2
lamduh_f = c / f_f

v_f = 2*np.pi*(L_f/2) * f_f
a_f = v_f**2 / (L_f / 2)
F = a_f * m_f

stress = F / (np.pi * r_f**2)

print('Ball radius = ' + '{:0.2f}'.format(r_f) + ' m')
print('Acceleration of ball = ' + '{:0.2g}'.format(a_f) + ' m/s**2')
print('Stress = ' + '{:0.2f}'.format(stress/sigma_yield) + 'x Yield Stress')

Ball radius = 40.00 m
Acceleration of ball = 2e+13 m/s**2
Stress = 0.81x Yield Stress


## <font color='Navy'>Radiation of a dumbell</font>

The dumbell is levitated from its middle point using a magnet. So we can spin it at any frequency without friction.

The quadrupole formula for the strain from this rotating dumbell is:

$\ddot{I} = \omega^2 \frac{M R^2}{2}$

The resulting strain is:

$h = \frac{2 G}{c^4 r} \ddot{I}$



In [69]:
r = 2 * lamduh   # take the distance to be 2 x wavelength

h   = (2*G)/(c**4 * r) * (1/2 * m_dumb * (L_dumb/2)**2) * (2*np.pi*f_rot)**2

print("Strain from a single (2018) dumbell is " + '{:0.3g}'.format(h))


r = 2 * lamduh_f   # take the distance to be 2 x wavelength

h_f = (2*G)/(c**4 * r) * (1/2 * m_f * (L_f/2)**2) * (2*np.pi*f_f)**2
d = c * 3600*24*365 * 1000

print("Strain from a single  (alien) dumbell  is " + '{:0.3g}'.format(h_f))
print("Strain from many (alien) dumbells is " + '{:0.3g}'.format(1e12*h_f*(r/d)) + ' at ' + str(1) + ' k lt-yr')

Strain from a single (2018) dumbell is 1.53e-41
Strain from a single  (alien) dumbell  is 7.75e-30
Strain from many (alien) dumbells is 2.66e-32 at 1 k lt-yr


## <font color='Navy'>Phased Array</font>

Beam pattern for a 2D grid of rotating dumbells

Treat them like point sources?

Make an array and add up all the spherical waves

In [ ]:
plt.figure()
for ii, lam in enumerate(lamList):
    lamStr = '{:.0e}'.format(lam)
    plt.semilogx(gList, snrFrac[ii], 'o', ms=20, alpha=0.6,
                 label='$\\lambda = {}\\times10^{{{}}}$ m'.format(*lamStr.split('e')))
plt.legend(loc=2, numpoints=1)
plt.grid()
plt.xlabel(r'Coupling $g = \delta_\mathrm{SM}\delta_\mathrm{DM}$')
plt.ylabel(r'Fraction with SNR > ' + str(snrThresh))
#plt.ylim([-0.05, 0.2])
plt.title('Yukawa-type DM clump interaction events in LISA')
plt.text(0.1, 0.18, 'Mass range: $10^6$ to $10^{10}$ kg')
plt.savefig('Figures/' + 'lisaYukawa.pdf')